<a href="https://colab.research.google.com/github/catafest/colab_google/blob/master/catafest_0059.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install with pip tool - python modules

In [45]:
from dataclasses import dataclass, field

symbols = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
           'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
           ' ', '!', '.', ',', '?', "'"]  # Define symbols outside the class

@dataclass
class HParams:
    # Experiment Parameters
    epochs: int = 500
    iters_per_checkpoint: int = 1000
    seed: int = 1234
    dynamic_loss_scaling: bool = True
    fp16_run: bool = False
    distributed_run: bool = False
    dist_backend: str = "nccl"
    dist_url: str = "tcp://localhost:54321"
    cudnn_enabled: bool = True
    cudnn_benchmark: bool = False

    ignore_layers: list = field(default_factory=list)  # Changed line    # Text Parameters
    text_cleaners: list = field(default_factory=lambda: ['english_cleaners'])  # Use default_factory
    # Mel parameters
    n_mel_channels: int = 80
    # n_mel_channels: int = 256
    # filter_length: int = 1024
    filter_length: int = 2048
    # hop_length: int = 256
    hop_length: int = 512
    # win_length: int = 1024
    win_length: int = 2048
    sampling_rate: int = 22050
    mel_fmin: float = 0.0
    mel_fmax: float = 8000.0

    # Model Parameters
    n_symbols: int = field(init=False)  # Indicate that n_symbols is calculated later
    symbols_embedding_dim: int = 512

    # Encoder parameters
    encoder_kernel_size: int = 5
    encoder_n_convolutions: int = 3
    encoder_embedding_dim: int = 512

    # Decoder parameters
    n_frames_per_step: int = 1  # currently only 1 is supported
    decoder_rnn_dim: int = 1024
    prenet_dim: int = 256
    max_decoder_steps: int = 1000
    gate_threshold: float = 0.5
    p_attention_dropout: float = 0.1
    p_decoder_dropout: float = 0.1

    # Attention parameters
    attention_rnn_dim: int = 1024
    attention_dim: int = 128

    # Location Layer parameters
    attention_location_n_filters: int = 32
    attention_location_kernel_size: int = 31

    # Mel-post processing network parameters
    postnet_embedding_dim: int = 512
    postnet_kernel_size: int = 5
    postnet_n_convolutions: int = 5

    # Optimization parameters
    use_saved_learning_rate: bool = False
    learning_rate: float = 1e-3
    weight_decay: float = 1e-6
    grad_clip_thresh: float = 1.0
    batch_size: int = 64  # set model's padded outputs to padded values

    def __post_init__(self):
        self.n_symbols = len(symbols)  # Calculate n_symbols after initialization


def create_hparams(hparams_string=None, verbose=False):
    """Create model hyperparameters. Parse nondefault from given string."""

    hparams = HParams()
    if hparams_string:
        print('Parsing command line hparams: %s', hparams_string)
        #hparams.parse(hparams_string) #Removed this line, as it was causing errors

    if verbose:
        print('Final parsed hparams: %s', hparams.values())#hparams.values() changed to vars(hparams)

    return hparams

In [46]:
!pip install librosa==0.9.2
!pip install --upgrade tensorflow

If you want to use next cell then you need to set runtime to T4 GPU.


In [47]:
project_name = "tacotron2"  # Înlocuiți cu calea corectă către proiect
tacotron2_pretrained_model = "tacotron2_statedict.pt"  # Înlocuiți cu calea corectă
waveglow_pretrained_model = "waveglow_old.pt"

In [48]:
%cd /content

gpu = !nvidia-smi --query-gpu=gpu_name --format=csv,noheader
print("Current GPU: " + gpu[0])

if gpu[0] == "Tesla K80" and optix_enabled:
  print("OptiX disabled because of unsupported GPU")
  optix_enabled = False

/content
Current GPU: Tesla T4


In [3]:
# @title Memory Information

import psutil
from IPython.display import HTML, display

def get_size(bytes, suffix="B"):
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor

# Retrieve memory information
svmem = psutil.virtual_memory()
total = get_size(svmem.total)
available = get_size(svmem.available)
used = get_size(svmem.used)
percentage = svmem.percent

# Create an HTML table with memory information (Rounded Corners variant with dark theme)
table_content = f"""
<table>
    <tr>
        <th colspan="2">Memory Information</th>
    </tr>
    <tr>
        <th>Total</th>
        <td>{total}</td>
    </tr>
    <tr>
        <th>Available</th>
        <td>{available}</td>
    </tr>
    <tr>
        <th>Used</th>
        <td>{used}</td>
    </tr>
    <tr>
        <th>Percentage</th>
        <td>{percentage}%</td>
    </tr>
</table>
"""

# Display the table
display(HTML(table_content))

Let's start

In [4]:
#@title
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/NVIDIA/tacotron2.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install
  !git clone -q --recursive {git_repo_url}
  !cd {project_name}/waveglow && git checkout 9168aea
  !pip install -q librosa unidecode
  !pip install -q --upgrade gdown




This need to fix with new changes , see : Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

In [49]:
import sys
sys.path.append(join(project_name, 'waveglow/'))
sys.path.append(project_name)
import time
import gdown
import matplotlib
import matplotlib.pylab as plt
plt.rcParams["axes.grid"] = False
#@title
tacotron2_pretrained_model = 'tacotron2_statedict.pt'
if not exists(tacotron2_pretrained_model):
  # download the Tacotron2 pretrained model
  gdown.download('https://drive.google.com/uc?id=1c5ZTuT7J08wLUoVZ2KkUs_VdZuJ86ZqA', tacotron2_pretrained_model, quiet=False)
waveglow_pretrained_model = 'waveglow_old.pt'
if not exists(waveglow_pretrained_model):
  # download the Waveglow pretrained model
  gdown.download('https://drive.google.com/uc?id=1WsibBTsuRg_SF2Z6L6NFRTT-NjEy1oTx', waveglow_pretrained_model, quiet=False)

FileURLRetrievalError: Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1c5ZTuT7J08wLUoVZ2KkUs_VdZuJ86ZqA

but Gdown can't. Please check connections and permissions.

In [50]:

!tf_upgrade_v2 --intree "/content/tacotron2" --outtree "/content/tacotron2_v2" --reportfile "/content/report.txt"


2025-04-13 13:19:46.397781: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744550386.416550   14982 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744550386.422217   14982 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744550386.436938   14982 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744550386.436965   14982 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744550386.436971   14982 computation_placer.cc:177] computation placer alr

In [51]:
#@title
import IPython.display as ipd
import numpy as np
import torch

from model import Tacotron2
from layers import TacotronSTFT
from audio_processing import griffin_lim
from text import text_to_sequence
from denoiser import Denoiser

def plot_data(data, figsize=(16, 4)):
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='bottom',
                       interpolation='none', cmap='viridis')

torch.set_grad_enabled(False)



This need to be fix with the new changes

In [53]:
# initialize Tacotron2 with the pretrained model
import torch
from model import Tacotron2

hparams = create_hparams()
hparams.sampling_rate = 22050
model = Tacotron2(hparams)

model.load_state_dict(torch.load(tacotron2_pretrained_model)['state_dict'])
_ = model.cuda().eval()  # .half()

# initialize Waveglow with the pretrained model
# waveglow = torch.load(waveglow_pretrained_model)['model']
# WORKAROUND for: https://github.com/NVIDIA/tacotron2/issues/182
import json
from glow import WaveGlow
waveglow_config = json.load(open('%s/waveglow/config.json' % project_name))['waveglow_config']
waveglow = WaveGlow(**waveglow_config)
waveglow.load_state_dict(torch.load(waveglow_pretrained_model)['model'].state_dict())
_ = waveglow.cuda().eval()#.half()
for k in waveglow.convinv:
    k.float()
denoiser = Denoiser(waveglow)

AttributeError: 'HParams' object has no attribute 'mask_padding'

In [37]:
INPUT_TEXT = "I speak about how Waveglow is really awesome!"

In [39]:
import json
from glow import WaveGlow

waveglow_config = json.load(open('%s/waveglow/config.json' % project_name))['waveglow_config']
waveglow = WaveGlow(**waveglow_config)
waveglow.load_state_dict(torch.load(waveglow_pretrained_model)['model'].state_dict())
_ = waveglow.cuda().eval()#.half()
for k in waveglow.convinv:
    k.float()
audio = waveglow.infer(mel_outputs_postnet, sigma=0.666)
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

/content/tacotron2/waveglow/glow.py:74: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at /pytorch/aten/src/ATen/native/BatchLinearAlgebra.cpp:2485.)
  W = torch.qr(torch.FloatTensor(c, c).normal_())[0]
/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


NameError: name 'waveglow_pretrained_model' is not defined

In [40]:
# remove waveglow bias
audio_denoised = denoiser(audio, strength=0.01)[:, 0]
ipd.Audio(audio_denoised.cpu().numpy(), rate=hparams.sampling_rate)

NameError: name 'denoiser' is not defined

In [41]:
#@title
sequence = np.array(text_to_sequence(INPUT_TEXT, ['english_cleaners']))[None, :]
sequence = torch.autograd.Variable(torch.from_numpy(sequence)).long()
sequence = sequence.cuda()

mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
plot_data((mel_outputs.data.cpu().numpy()[0],
           mel_outputs_postnet.data.cpu().numpy()[0],
           alignments.data.cpu().numpy()[0].T))

NameError: name 'model' is not defined